#Description

##Background: Equity in Healthcare
Healthcare inequity is a global challenge. Addressing this challenge has an extensive positive impact on women’s health, which is key for societies and economies to thrive. This datathon is designed to help discover whether disparate treatments exist and to understand the drivers of those biases, such as demographic and societal factors.

In the first datathon challenge we explored the relationship between socio economic aspects that contribute to health equity. For this next challenge we’re building on that analysis to see how climate patterns impact access to healthcare.

##Overview: The Dataset and Challenge
Gilead Sciences is the sponsor for the 2024 WiDS Datathon.The dataset originated from Health Verity, one of the largest healthcare data ecosystems in the US. It was enriched with third party geo-demographic data to provide views into the socio economic aspects that may contribute to health equity. For this challenge, the dataset was then further enriched with zip code level climate data.

##Challenge task:
Predicting the duration of time it takes for patients to receive metastatic cancer diagnosis.

###Why is this important?
Metastatic TNBC is considered the most aggressive TNBC and requires urgent and timely treatment. Unnecessary delays in diagnosis and subsequent treatment can have devastating effects in these difficult cancers. Differences in the wait time to get treatment is a good proxy for disparities in healthcare access.

The **primary goal** of building these models is to detect relationships between demographics of the patient with the likelihood of getting timely treatment. The **secondary goal** is to see if climate patterns impact proper diagnosis and treatment.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
# import statsmodels.api as sm
# from sklearn.preprocessing import LabelBinarizer

# Importing Data and Initial Data exploration

In [ ]:
train = pd.read_csv('/kaggle/input/widsdatathon2024-challenge2/train.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2024-challenge2/test.csv')
solution_template = pd.read_csv('/kaggle/input/widsdatathon2024-challenge2/solution_template.csv')
# shuffle the training set - helps prevent bias during training
train = train.reindex(np.random.permutation(train.index))
display(train.head(5))

In [ ]:
# Let's print out the size of the dataset
print("Number of rows and columns on training set are: ", train.shape)
print("Number of rows and columns on test set are: ", test.shape)

In [ ]:
display(solution_template.head(5))
print(solution_template.shape)

In [ ]:
# Let's print a concise summary of train dataset.
train.info()

From the summary, we see that we have 11 categorical features

In [ ]:
# Let's print a concise summary of test dataset.
test.info()

In [ ]:
#Let's print out the categorical columns and the numerical columns

#Categorical:
cat_cols = [col for col in train.columns if train[col].dtype == 'object']
# print('Categorical Columns: ')
# for value in cat_cols:
#     print(value)
print('Categorical Columns: ', cat_cols)

#Numerical:
num_cols = []
for col in train.columns:
  if train[col].dtype == 'int64' or 'float64': #(you could also use != 'object')
    num_cols.append(col)
print('Numerical Columns: ')
for val in num_cols:
    print(val)


Let's find the number of unique values in each feature

In [ ]:
unique_values = pd.concat(
    [
        train.drop('metastatic_diagnosis_period', axis=1).nunique().rename('train'),
        test.nunique().rename('test')
    ],
    axis=1
)

display(unique_values.T)

# Plotting categorical features

In [ ]:
#Let's plot some of the categorical columns

# Creating a barplot for 'payer_ type'

# Setting the width and height of the figure
plt.figure(figsize=(9, 7))

# Bar chart
bars = train['payer_type'].value_counts().plot(kind='bar')

# Setting the value count and fontsize of each bar labels (2103, 882...)
bars.bar_label(bars.containers[0],fontsize = 13)

# Setting the size of the xticks, yticks, and rotating the tick labels to be horizontal
bars.tick_params(labelsize = 10, labelrotation = 360)
plt.show()

We see that the number of people using medicare advantage and medicaid are pretty similar. Over half of the patients used commercial payments. Would be nice to explore what commercial here means. Does it mean cash or company sponsored insurances like Anthem and Blue Cross Blue shield?

In [ ]:
# Creating a Bar Chart of Patient Race

# Setting the width and height of the figure
plt.figure(figsize=(12, 6))

# Bar chart for train data
plt.subplot(121)
bars = train['patient_race'].value_counts().plot(kind='bar')
# You can also use seaborn to create the bars:
# bars = sns.barplot(y = train.index, x = train['patient_race'])
# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patients by Race in Train Data",fontsize = 14)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)

# Bar chart for test data
plt.subplot(122)
bars = test['patient_race'].value_counts().plot(kind='bar')
# You can also use seaborn to create the bars:
# bars = sns.barplot(y = train.index, x = train['patient_race'])
# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patients by Race in Test Data",fontsize = 14)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)

plt.tight_layout()
plt.show()

Patient race data is heavily skewed with the number of patients identifying as white exceeding all other races combined.

In [ ]:
# Creating a Bar Plot of Patient State

# Setting the width and height of the figure
plt.figure(figsize=(12, 9))

# Bar chart
bars = train['patient_state'].value_counts().plot(kind='bar')

# # Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 8)

# # Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patients by State in Train Data",fontsize = 15)
# bars.set_xlabel("Patient Race",fontsize = 15)
# bars.set_ylabel("Number of Patients",fontsize = 15)
# # Setting the size of the xticks and yticks
# bars.tick_params(labelsize = 10, labelrotation = 360)
plt.show()

From the barplot, California(CA) has the highest number of patients followed by New York(NY) and Texas(TX).

One reasonable explanation could be because they are the most populated states.

It would also be great to look into other factors such as food quality, air quality etc. that may explain why these three states have the highest number of cancer patients.

In [ ]:
# Creating a Bar Plot of Patient Region

# Setting the width and height of the subplots
plt.figure(figsize=(12, 6))

# Bar chart for train data
plt.subplot(121) #(using plt.subplot to create subplots)
bars = train['Region'].value_counts().plot(kind='bar')
# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patients by Region in Train Data",fontsize = 14)
bars.set_xlabel("Patient Region",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)

# Bar chart for test data
plt.subplot(122)
bars = test['Region'].value_counts().plot(kind='bar')
# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patients by Region in Test Data",fontsize = 14)
bars.set_xlabel("Patient Region",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)

plt.tight_layout()
plt.show()

Northeast region seems to have the least amount of cancer patients. The data doesn't describe exactly what states are considered to be in each region, so it can be hard to draw further value from the plot above.

In [ ]:
# Plotting the distribution of patient races by region

# Since the patient race is a categorical feature, we'll try using a countplot variation to plot the distribution.
# A count plot can be thought of as a histogram across a categorical, instead of quantitative, variable.
# The basic API and options are identical to those for barplot(), so you can compare counts across nested variables.

# Setting the width and height of the subplots
plt.figure(figsize=(15, 9))

# Plot for train data
plt.subplot(211) #(no. of rows = 2, no. of cols = 1, plot no.1)
bars = sns.countplot(x = train['patient_race'], hue = train['Region'], palette = ['blue', 'deeppink','darkviolet','lime'])

# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
bars.bar_label(bars.containers[1],fontsize = 10)
bars.bar_label(bars.containers[2],fontsize = 10)
bars.bar_label(bars.containers[3],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patient Race by Region(Train)",fontsize = 14)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)


# Plot for test data
plt.subplot(212) #(no. of rows = 2, no. of cols = 1, plot no.2)
bars = sns.countplot(x = test['patient_race'], hue = test['Region'], palette = ['blue', 'deeppink','darkviolet','lime'])

# Setting the size of inside bar labels
bars.bar_label(bars.containers[0],fontsize = 10)
bars.bar_label(bars.containers[1],fontsize = 10)
bars.bar_label(bars.containers[2],fontsize = 10)
bars.bar_label(bars.containers[3],fontsize = 10)
# Setting the title, xlabel, ylabel for the plot and their sizes
bars.axes.set_title("Number of Patient Race by Region(Test)",fontsize = 14)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)
# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 360)

plt.tight_layout()
plt.show()

# Plotting numerical features

In [ ]:
# Plotting some numerical features

# Creating a Histogram for Patient Age

# Create subplots(using plt.subplots) with 1 row and 2 columns and their width and height(figsize)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
# Plot for patient_age in train data
axes[0].hist(train['patient_age'], bins=15, color='skyblue', edgecolor='black')
# Setting the title, xlabel, ylabel for the plot and their sizes
axes[0].set_title('Patient Age Distribution in Train Data', fontsize = 14)
axes[0].set_xlabel('Patient Age (yrs)', fontsize = 12)
axes[0].set_ylabel('Frequency', fontsize = 12)

# Plot for patient_age in test data
axes[1].hist(test['patient_age'], bins=15, color='hotpink', edgecolor='black')
# Setting the title, xlabel, ylabel for the plot and their sizes
axes[1].set_title('Patient Age Distribution in Test Data', fontsize = 14)
axes[1].set_xlabel('Patient Age (yrs)', fontsize = 12)
axes[1].set_ylabel('Frequency', fontsize = 12)


plt.tight_layout()
plt.show()


In [ ]:
# Income distribution

# Create subplots(using plt.subplots) with 1 row and 2 columns and their width and height(figsize)
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
# Plot for patient_age in train data
# axes[0].hist(train['patient_age'], bins=15, color='skyblue', edgecolor='black')
income_median_train = train['income_household_median'].median()
axes[0].hist(train['income_household_median'], bins=15, color='skyblue', edgecolor='black')

# Setting the title, xlabel, ylabel for the plot and their sizes
axes[0].set_title('Median Income Distribution in train data', fontsize = 14)
axes[0].set_xlabel('Household Income($)', fontsize = 12)
axes[0].set_ylabel('Frequency', fontsize = 12)

# Plot for patient_age in test data
income_median_test = test['income_household_median'].median()
axes[1].hist(test['income_household_median'], bins=15, color='hotpink', edgecolor='black')
# Setting the title, xlabel, ylabel for the plot and their sizes
axes[1].set_title('Median Income Distribution in test data', fontsize = 14)
axes[1].set_xlabel('Household Income($)', fontsize = 12)
axes[1].set_ylabel('Frequency', fontsize = 12)
plt.tight_layout()
plt.show()


# More Exploratory Analysis

In [ ]:
train['patient_gender'].info()

From the info above, we see that all patients identified as Female. So we do not need to plot a chart for the patient gender.

Let's take a deeper look into the metastatic diagnosis period

In [ ]:
train['metastatic_diagnosis_period'].describe()

In [ ]:
# Distribution of the metastatic diagnosis period feature
plt.figure(figsize = (10, 6))
target_dist = train['metastatic_diagnosis_period'].plot(kind='hist', figsize=(10, 4), xlabel='metastatic_diagnosis_period', ylabel='Count')

Let's look at the mean and median of this feature for each race

In [ ]:
# Let's find the count for when the metastatic diagnosis period was zero days

zero_count = (train['metastatic_diagnosis_period'] == 0).sum()
print("The amount of metastatic diagnoses that took 0 days are: ", zero_count)

In [ ]:
# Let's take this a step further, and check zero_count for each patient race

# Creating a new dataframe where metastatic diagnosis period is 0 days.
zero_days_train = train[train['metastatic_diagnosis_period'] == 0]

# Plotting this in relation to patient race
plt.figure(figsize=(10, 6)) # Setting the size of the plot figure

bars = zero_days_train['patient_race'].value_counts().plot(kind='bar')
bars.bar_label(bars.containers[0],fontsize = 10)
bars.axes.set_title("No. of Patients with diagnosis period of 0 days",fontsize = 14)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Number of Patients",fontsize = 12)

plt.show()

We see that white patients have the quickest metastatic diagnosis period compared to other races.

This could also be because white patient population is the largest amongst other race populations.

In [ ]:
# Mean metastatic diagnosis period for each race

# Setting the size of the plot figure
plt.figure(figsize=(9, 6))

# Plotting the bar graph
ax = sns.barplot(data=train, x='patient_race', y='metastatic_diagnosis_period',  errorbar = None)  # ci=None disables error bars
plt.xlabel('Patient Race')
plt.ylabel('Mean Metastatic Diagnosis Period(in days)')
plt.title('Mean Metastatic Diagnosis Period by Patient Race (Train)')
plt.xticks(rotation = 360)  # Rotating x-axis labels for better readability

# Annotate each bar with its corresponding mean value
for bar in ax.patches:
    ax.annotate(format(bar.get_height(), '.2f'),
                 (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                 ha ='center', va ='center',
                 size = 10, xytext = (0, 8),
                 textcoords = 'offset points')

plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()

In [ ]:
# Getting a summary of the 'breast_cancer_diagnosis_code' feature
print("Statistical summary of 'breast_cancer_diagnosis_code' \n")
train['breast_cancer_diagnosis_code'].info()
print("\nFirst five rows:")
train['breast_cancer_diagnosis_code'].head()


In [ ]:
# Let's look at the Top 10 metastatic diagnosis codes

code_counts = train['metastatic_cancer_diagnosis_code'].value_counts()
top_10_codes = code_counts.head(10)
# print("Top 10 Metastatic Diagnosis Codes:")
# print(top_10_codes)

plt.figure(figsize = (10, 8))
ax = sns.barplot(x = top_10_codes.index,hue = top_10_codes.index,y = top_10_codes.values, palette = 'viridis')
for i in range(10): # since we are only showing top 10 codes.
  ax.bar_label(ax.containers[i], fontsize = 10) # prints the count of each bar/container.
plt.title('Top 10 Metastatic Cancer Codes', fontsize = 14)
plt.xlabel('Metastatic Cancer Code', fontsize = 12)
plt.ylabel('Count', fontsize = 12)
plt.show()

# Correlation between Features

In [ ]:
# Let's start with patient race vs payer type

# Plot for Train Data

# We'll create a daframe by crossing patient race and payer type using crosstab function in pandas then plot the dataframe
bars = pd.crosstab(train['patient_race'],train['payer_type']).plot(kind = "bar",stacked = True, color =['#24b1d1', '#ae24d1', '#00FF00'] ) #stacked ensures we plot them ontop of each other

# Setting the labels inside the stacked bars
# Since we have stacked bars, we will use the 3 lines below to find counts of types of paymemnt in each race
# Alternatively, one can use a for loop to pass through containers and save count.
bars.bar_label(bars.containers[0],fontsize = 7, label_type= 'center') #counts on commercial
bars.bar_label(bars.containers[1],fontsize = 7, label_type= 'center', color = 'white') #counts on medicaid
bars.bar_label(bars.containers[2],fontsize = 7, label_type= 'center', color = 'red') #counts on medicare advantage

# Setting the title and xlabel for the plot and their sizes
bars.axes.set_title("Types of payments by Patient Race in Train Data",fontsize = 13)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Count",fontsize = 12)

# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 0)

plt.show()

From the plot above, we first acknowledge that the white race population is the highest, so their payment type counts will overall be larger than the other patients.

White patients seem to be fairly distributed between the 3 payment options with more using medicare advantage.

Black patients more significantly use medicaid as the form of payment rather than Medicare Advantage and Commercial.

Asian patients use commercial and medicare at similar rates with the least being medicare advantage.

Hispanic patients use medicare about twice as much as they use commercial or medicare advantage as forms of payment.

Othe patients use commercial about 4 times as much as they use medicare and about 6 times as much as medicare advantage.

Interestingly, we can also point out that only white patients use medicare advantage more than the other payments Whereas Asian, Black and Hispanic patients use medicaid the most.

In [ ]:
# Plot for Test Data

# We'll create a daframe by crossing patient race and payer type using crosstab function in pandas then plot the dataframe
bars = pd.crosstab(test['patient_race'],train['payer_type']).plot(kind = "bar",stacked = True, color =['#24b1d1', '#ae24d1', '#00FF00'] ) #stacked ensures we plot them ontop of each other

# Setting the labels inside the stacked bars
bars.bar_label(bars.containers[0],fontsize = 7, label_type= 'center') #counts on commercial
bars.bar_label(bars.containers[1],fontsize = 7, label_type= 'center', color = 'white') #counts on medicaid
bars.bar_label(bars.containers[2],fontsize = 7, label_type= 'center', color = 'red') #counts on medicare advantage

# Setting the title and xlabel for the plot and their sizes
bars.axes.set_title("Types of payments by Patient Race in Test Data",fontsize = 13)
bars.set_xlabel("Patient Race",fontsize = 12)
bars.set_ylabel("Count",fontsize = 12)

# Setting the size of the xticks and yticks
bars.tick_params(labelsize = 10, labelrotation = 0)

plt.show()

In test data plot above, we see that commercial is the most used form of payment across all patient races.

We know that socio-economic factors may play an impact into  ones' quality of health from the food they are likely to consume to the health services available to them. Let's plot a correlation heatmap to analyze the relationship between income, poverty and unemployment.

In [ ]:
# Correlastion Heatmap
correlation_map_variables = ['income_household_median', 'poverty', 'unemployment_rate']

data = train[correlation_map_variables]

corr_matrix = data.corr()

plt.figure(figsize = (8,6))
sns.heatmap(corr_matrix, annot=True, cmap="BuPu", vmin=-1, vmax=1)
plt.title('Correlation Heatmap for Income, Unemployment Rate, and Poverty')
plt.show()

Poverty in our dataset is measured by the median value of owner occupied homes.

Household income and Poverty have a high correlation of -0.69, showing what we know that the higher the household income, less likely the poverty.

Similarly, Unemployment Rate and Poverty have a high correlation of 0.68, showing that unemployed people are more likely to fall into poverty.


Seeing the presence of multi collinearilty between these features,we'll drop poverty from our dataset.

In [ ]:
# Next we'll look at the relationship btw bmi and patient races.
# Before we plot, it's important to note that both of these features have over 50% missing values, so the plot
# insights may be inaccurately represented.

# train[['patient_race','bmi']].mean().plot(kind='bar', color="red")
# train.groupby(['patient_race'])["bmi"].head(5)
# race_bmi = train[["patient_race", "bmi"]]
# race_bmi.plot(kind = "bar")
# race_bmi = train[(train['patient_race']!= 'NaN') & (train['bmi']!= 'NaN')]
# race_bmi_updated = race_bmi[["patient_race", "bmi"]]
# race_bmi_updated.plot(kind = "bar")

In [ ]:
# More correlations that could be further explored

# poverty vs patient_race
# income_household vs metastatic_diagnosis_period
# health_uninsured vs metastatic_diagnosis_period
# limited_english vs metastatic_diagnosis_period
# patient_age vs metastatic_diagnosis_period

## Missing Values

In [ ]:
# Let's check for missing values in our dataset. Any changes we implement on train set on this section, we'll also implement on test set.

missing_values = ( (train.isna().sum()) / len(train) ) * 100 # gives us percentage of missing values in each column
missing_values = missing_values.drop(missing_values[missing_values == 0].index) # exclude columns with 0 missing values
missing_values = missing_values.sort_values(ascending = False) # Sort missing values in descending order
missing_values.head(15)

In [ ]:
print("Features with over 10% missing values are: \n", missing_values[missing_values > 10])

We see that metastatic first novel treatment has over 99% missing values. With that much data missing, the 2 metastatic features provide close to zero valuable info. Therefore, we will be deleting them

bmi is a valuable feature, but it has over 68% missing values. We will take a closer look to see if we will choose to keep it and do feature engineering or drop it.

For the rest (patient race, payer type, etc..) we will be replacing missing values with averages.

Let's look at the percentage of missing values by columns in both train and test sets

In [ ]:
pct_missing = pd.concat(
    [
        (train.drop('metastatic_diagnosis_period', axis=1).isna().sum() / len(train)).rename('train'),
        (test.isna().sum() / len(test)).rename('test')
    ],
    axis=1
)

with pd.option_context('display.precision', 4):
    display(pct_missing.T)

# Data Preprocessing

We'll start by deleting features with the most missing values. From the code above, we see that metastatic_first_novel_treatment and metastatic_first_novel_treatment_type have over 99% missing values. We won't derive any value from them.

We also saw that patient gender has only one unique value(female), so we'll remove it as well since it doesn't derive much insight.

In [ ]:
print("Shape before dropping columns: \n")
print("train shape", train.shape)
print("test shape", test.shape)

In [ ]:
df_train = train.drop(columns = ['patient_gender', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'])
df_test = test.drop(columns = ['patient_gender', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'], axis=1)

In [ ]:
print("Shape after dropping columns: \n")
print("df_train shape: ", df_train.shape)
print("df_test shape: ", df_test.shape)

Next, we will check for any duplicated rows in our dataset. If we find any, we'll drop the rows.



In [ ]:
duplicate_train = df_train[df_train.duplicated() == True]
duplicate_test = df_test[df_test.duplicated() == True]
print(duplicate_train.shape)
print(duplicate_test.shape)

We see both train and test sets do not have any duplicate rows

Now, we'll detect and fix outliers/inconsistencies. They appear in the following ways:



*   **Location** based features: patient_zip3, patient_state, Division, Region
<!-- * **Population** based features: population, veteran -->
*   **Temperature** based features: Average of Jan-13, Average of Feb-13...Average of Dec-18
*   Patient **Diagnosis** feature: breast_cancer_diagnosis_code







Let's start by checking if there's more than one state per patient_zip.

Test Set

In [ ]:
df_test[['patient_state','patient_zip3']].drop_duplicates().groupby('patient_zip3').count().sort_values('patient_state', ascending=False).head(10)

Train Set

In [ ]:
df_train[['patient_state','patient_zip3']].drop_duplicates().groupby('patient_zip3').count().sort_values('patient_state', ascending=False).head(10)

We see that zip 630 and zip 864 are in 2 states

Let's take a step further and see what these states are for each of the zipcodes. We'll use df.loc property to access a group of rows and columns for the 2 zipcodes.

Using df.loc, we'll create a conditional that returns a boolean series with column labels specified

In [ ]:
df_train.loc[df_train['patient_zip3'].isin([630, 864]), ['patient_state', 'patient_zip3']].drop_duplicates()

So zip 630 shows up in both MO and IL, whereas zip 864 shows up in both AZ and CA.

According to [united states zipcodes](https://www.unitedstateszipcodes.org/) , zip 630 belongs to MO and zip 864 belongs to AZ.

We'll fix the data to reflect that bu using np.where

In [ ]:
df_train['patient_state'] = np.where(df_train['patient_zip3'] == 630, 'MO', df_train['patient_state'])
df_train['patient_state'] = np.where(df_train['patient_zip3'] == 864, 'AZ', df_train['patient_state'])

Let's look at state allocation to division. In reality, each state belongs to a single division. Let's check for any inconsistencies

Test Data

In [ ]:
df_test[['patient_state','Division']].drop_duplicates().groupby('patient_state').count().sort_values('Division', ascending=False).head(10)

Train set

In [ ]:
df_train[['patient_state','Division']].drop_duplicates().groupby('patient_state').count().sort_values('Division', ascending=False).head(10)

Let's look at the 2 divisions where MO appears

In [ ]:
df_train.loc[df_train['patient_state'] == 'MO', ['patient_state', 'Division']].drop_duplicates()

Missouri(MO) seems to appear in both West North Central and East North Central.

Let's count the values for each division

In [ ]:
df_train.loc[df_train['patient_state'] == 'MO'].value_counts('Division')

Since all but 1 are in West North Central, we'll assume it was an error for the single one to be in East North CentraL. So, we'll change it to West North Central.

In [ ]:
df_train['Division'] = np.where(df_train['patient_state'] == 'MO', 'West North Central', df_train['Division'])

In [ ]:
df_train.loc[df_train['patient_state'] == 'MO', ['patient_state', 'Division']].drop_duplicates()

In [ ]:
df_train[['patient_state','Division']].drop_duplicates().groupby('patient_state').count().sort_values('Division', ascending=False).head(5)

Let's also check for 'region'.

In [ ]:
# Test Set
df_test[['patient_state', 'Region']].drop_duplicates().groupby('patient_state').count().sort_values('Region', ascending=False).head(5)

In [ ]:
# Train Set
df_train[['patient_state','Region']].drop_duplicates().groupby('patient_state').count().sort_values('Region', ascending=False).head(5)

Region feature is consistent in both train and test sets. No need for any changes

Next features we'll explore are Temperature related.

In [ ]:
df_train['Average of Jan-13'].head()

Let's get a monthly average instead of day averages in each month. This will reduce the number of features in our data set, and make training the ML model more efficient. We should see a decrease in no. of features after this.

In [ ]:
df_train.shape

In [ ]:
# creating a list with 12 months

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Iterating from each month to get average monthly temperature in train
for i in months:
    cols_train = [col for col in df_train.columns if col.startswith(f'Average of {i}')] # putting all day average temperatures of a month in the list
    cols_test = [col for col in df_test.columns if col.startswith(f'Average of {i}')]
    df_train[f'Average of {i}'] = df_train[cols_train].sum(axis = 1)/len(cols_train) # adding all values in cols divide by length to get mean monthly temp.
    df_test[f'Average of {i}'] = df_test[cols_test].sum(axis = 1)/len(cols_train)
    df_train.drop(cols_train, axis = 1, inplace = True) # dropping those columns whose average are taken above
    df_test.drop(cols_test, axis =1, inplace = True)

In [ ]:
new_temp_cols = [col for col in df_train.columns if col.startswith('Average of')]
new_temp_colss = [col for col in df_test.columns if col.startswith('Average of')]
print("Train Set: \n",new_temp_cols)
print("Test Set: \n",new_temp_colss)

In [ ]:
df_train.shape

Let's now check for inconsistencies with the breast cancer diagnosis.

Train

In [ ]:
df_train.groupby(['breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc'], as_index=False)['patient_id'].count()

Keep in mind that the last column is no. of patients in each diagnosis code rather than the patient id itself.

We see that there are some diagnosis codes that refer to male cancers despite all our patients being women as seen in patient_gender feature.

We'll convert the make diagnosis codes to their respective matching female diagnosis codes.


In [ ]:
# Fixing  inconsistencies by recoding male/unspecified codes to female codes
df_train['breast_cancer_diagnosis_code'] = df_train['breast_cancer_diagnosis_code'].replace({
    # Recode male to female
    'C50122': 'C50112', 'C50221': 'C50211', 'C50421': 'C50411', 'C50922': 'C50912',

    'C509': 'C5091'
})

Let's do the same check in test set

In [ ]:
df_test.groupby(['breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc'], as_index = True)['patient_id'].count()

There are no inconsistencies with the diagnosis code in our test set.

Now, let's work on **replacing missing values** in our train and test sets

In [ ]:
print("Missing values count in Train: ")
df_train.isnull().sum().sort_values(ascending = False).head(25) # counting the number of missing values in aech feature

In [ ]:
print("Missing values count in Test: ")
df_test.isnull().sum().sort_values(ascending = False).head(20)

Apart from the 3 features with the most missing values, we see that other features especially in train set have a couple missing values(5 and under).

Let's look into this by starting with **population based featured** and find ways to fill in the missing values.

For example: **'limited_english'**

In [ ]:
df_train[df_train.limited_english.isna()]

Looks like all the patients with missing values in limited english are from patient zip 772.

Let's get a subset with patients on zip 772 and see if there are any valid values.

In [ ]:
print(df_train[df_train.patient_zip3 == 772].limited_english.unique())
df_train[df_train.patient_zip3 == 772].head(10)

So it seems no patient in zip 772 has valid values in limited_english.

We also see that only 5 patients in our dataset are in zip 772. Let's see if we can leverage data from the same state(Texas)

In [ ]:
df_train[(df_train.patient_state == 'TX') & (~df_train.limited_english.isna())].head()

Let's reduce this into population related features so we can get a better look at the values.

In [ ]:
df_train.shape

In [ ]:
popln_cols = df_train.loc[:, 'population':'veteran'].columns.to_list()
df_popln = df_train[['patient_zip3', 'patient_state'] + popln_cols].drop_duplicates().sort_values('patient_zip3')
print(df_popln.shape)
df_popln.head()

Let's run the previos code to see limited_english values in Texas.

In [ ]:
df_popln[(df_popln.patient_state == 'TX') & (~df_popln.limited_english.isna())].head()

We see that there are values in limited_english feature. We could replace missing values with averages from the state or with values from the closest zip.

In [ ]:
df_popln[(df_popln.patient_state == 'TX') & (df_popln.patient_zip3 > 767)].head(10)

Replacing missing values from 772 with those of nearest zips(770 or 773) may be riskier because sometimes there's a huge difference in population statistics between different zipcodes.

For example the population varies significantly btw 770 (33353.72	) vs 772 ( 4459.00) as well as btw 773 (24751.20) and 772 ( 4459.00).

So, replacing with average for the state is a better move.

Before doing that, let's check a couple more features to see if other missing values are also from the same zip.

In [ ]:
df_train[df_train.home_ownership.isna()]

In [ ]:
df_train[df_train.family_size.isna()]

In [ ]:
df_train[df_train.rent_median.isna()]

We can assume a lot of the population based missing values are from patients located in zip 772.

We'll replace missing values in all population based features with the mean from the state.

In [ ]:
# Replace missing values

# creating a function
def mixed_replacement(df, group_col):
    for column in df.columns:
        if column != group_col:  # Exclude the group column
            # If the column is numerical, then replace missing values with mean
            if df[column].dtype in [np.dtype('float_'), np.dtype('int_')]:
                mean_impute = df.groupby(group_col)[column].mean()
                df[column] = df[column].fillna(df[group_col].map(mean_impute))

            # If the column is categorical, then replace missing values with mode
            else :
                mode_impute = df.groupby(group_col)[column].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
                df[column] = df[column].fillna(df[group_col].map(mode_impute))

    return df

# Impute missing values
df_popln = mixed_replacement(df = df_popln, group_col='patient_state')

In [ ]:
df_popln.isnull().sum().sort_values(ascending = False).head()

In [ ]:
print(df_popln.shape)

Let's tackle missing values in **payer_type**

About 13% of values are missing from payer_type. Let's quickly look at the distribution of the unique values in both train and test set.

In [ ]:
df_train['payer_type'].value_counts(dropna=False, normalize=True) # using normalize to return proportions rather than frequency

In [ ]:
df_test['payer_type'].value_counts(dropna=False, normalize=True)

From these distributions, a missing value could either indicate that the patient is either **uninsured** or using **commercial**(since it has the highest distribution)

In [ ]:
# For simplicity, let's assume missing values are 'COMMERCIAL'. We'll replace NaN with 'COMMERCIAL'

df_train['payer_type'] = df_train['payer_type'].fillna('COMMERCIAL')
df_test['payer_type'] = df_test['payer_type'].fillna('COMMERCIAL')

In [ ]:
df_train.isnull().sum().sort_values(ascending = False).head(25)

In [ ]:
# Let's merge the df_popln back into train and test sets.

# To ensure that the number of rows remains the same after the merge,we will remove 
# the duplicates from df_popln, keeping only unique combinations of patient_zip3 and patient_state.
df_popln_unique = df_popln.drop_duplicates(subset=['patient_zip3', 'patient_state'])

clean_train = df_train.drop(popln_cols, axis = 1).merge(
    df_popln_unique, how ='left', on =['patient_zip3', 'patient_state']
)
clean_test = df_test.drop(popln_cols, axis = 1).merge(
    df_popln_unique, how ='left', on =['patient_zip3', 'patient_state']
)

print(clean_train.shape, df_train.shape, clean_test.shape,  df_test.shape)
clean_train.head(10)

In [ ]:
clean_train.isnull().sum().sort_values(ascending = False).head()

In [ ]:
train.shape

In [ ]:
clean_train.shape

We still have to work something on 'bmi' and 'patient_race' regarding the missing values.

**bmi** - We'll replace missing values in bmi with 0

**patient_race** - We'll replace missing values with 'N/A'

In [ ]:
# Replacing all Nan values in 'bmi' with 0
clean_train['bmi'] = clean_train['bmi'].fillna(0)
clean_test['bmi'] = clean_test['bmi'].fillna(0)

# Replacing all Nan values in 'patient_race' with 'N/A'

clean_train['patient_race'] = clean_train['patient_race'].fillna('N/A')
clean_test['patient_race'] = clean_test['patient_race'].fillna('N/A')

In [ ]:
# Let's check if there are still any missing values in our data set
clean_train.isnull().sum().sort_values(ascending = False).head()


In [ ]:
clean_test.isnull().sum().sort_values(ascending = False).head(40)

Mmh, looks like we still have some missing values in the test set. We'll work on this first before moving to the next stage

In [ ]:
import warnings
def ignore_warning():
    # Code that generates warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ignore_warning()
    
# Saving all columns with null value
na_cols = clean_test.columns[clean_test.isna().sum() > 0].tolist()

# Filling these columns with null value with mean of each column
for column in na_cols:
    clean_test[column].fillna(clean_test[column].mean(), inplace=True)

# checking null values presence after imputation of mean
print(clean_test.isnull().sum().sort_values(ascending = False).head())
print(clean_test.columns[clean_test.isna().sum() > 0])

# Feature Engineering

After pre processing our data by filling in missing values and replacing outliers, we'll now do some feature engineering to make sure all our data is readable/recognizable by our machine learning model. Here's a summary of what we'll do:



*   Create age ranges of equal sizes ex: under 29, 30-39, 40-49 etc.
*   Convert other categorical features using suitable encoding method


Encoding categorical variables means transforming the categorical variables(be it in text or numerical form) into a numerical format that is compatible with machine learning algorithm.

Some widely used label encoding methods are:

1.   **One-hot Encoding** - represents categorical variables as binary vectors, which  increases dimensionality of the data set. For each category in a categorical column, a new binary column is created. The binary column will have a value of 1 if the class is present, else it will be zero.
2.   **Label Encoding** - converts the unique labels of a categorical variable into numeric form without affecting the dimensionality of the data set.
3.   **Ordinal Encoding** - this is similar to label encoding but allows you to explicitly define the mapping between categories and integer labels. This is especially useful when there is a clear and predefined ordinal relationship.
4.   **Count/Frequency Encoding** - replaces each category with the count of how many times it appears in the dataset. This encoding technique can be useful when there’s a correlation between the frequency of a category and the target variable. Also applicable for categorical features with a lot of categories. The count_encoder should be fit only on the train dataset. The fitted object should be used to transform test and out of time (OOT) datasets.
5.   **Target(Mean) Encoding** - involves replacing each category with the mean (or some other statistic) of the target variable for that category.

Note:

The main differences between Nominal Data and Ordinal Data are: 

While Ordinal Data has some predetermined or natural order, Nominal Data doesn't; it usually has some categories like if an animal is a mammal or reptile etc. Nominal data is qualitative or categorical data, while Ordinal data is considered “in-between” qualitative and quantitative data.

Learn more about these and other methods [here](https://medium.com/anolytics/all-you-need-to-know-about-encoding-techniques-b3a0af68338b).


In [ ]:
# putting patient age ranges into bin using pd.cut function
# pd.cut segments and sorts data values into bins. It is also useful for going from a continuous variable to a categorical variable
clean_train['age_group'] = pd.cut(clean_train['patient_age'], right=False, bins=[0, 30, 40, 50, 60, 70, 80, 90, np.inf], labels=[0,1,2,3,4,5,6,7]).astype(int)
clean_test['age_group']  = pd.cut(clean_test['patient_age'], right=False, bins=[0, 30, 40, 50, 60, 70, 80, 90, np.inf], labels=[0,1,2,3,4,5,6,7]).astype(int)

In [ ]:
# Let's check if our pd.cut function worked
clean_train['age_group'].value_counts()

In [ ]:
# Let's drop the 'breast_cancer_diagnosis_desc' and 'patient_id' columns since they won't be helpful in model training

# Dropping 'breast_cancer_diagnosis_desc'
clean_train.drop('breast_cancer_diagnosis_desc', axis = 1, inplace = True)
clean_test.drop('breast_cancer_diagnosis_desc', axis = 1, inplace = True)
# Dropping 'patient_id'
clean_train.drop('patient_id', axis = 1, inplace = True)
clean_test.drop('patient_id', axis = 1, inplace = True)

In [ ]:
# Encoding BMI into buckets
# Since we already put 0 for missing values in bmi, we'll start with label 1 for 0 < 'bmi' < 18.5

# clean_train['bmi_range'] +=  np.where(0 < clean_train['bmi'] < 18.5, 1)
# clean_train['bmi_range'] += np.where(18.5 <= clean_train['bmi'] < 25, 2)
# clean_train['bmi_range'] += np.where(25 <= clean_train['bmi'] < 30, 3)
# clean_train['bmi_range'] += np.where(clean_train['bmi'] >= 30, 4)


In [ ]:
# Let's standardize numerical columns and encode the categorical variables we just saw above

# First, let's find all the numerical columns
numeric_columns = clean_train.drop(['metastatic_diagnosis_period'], axis=1).select_dtypes(include = ['number']).columns
print("Numeric columns:\n", numeric_columns)
numeric_columns_test = clean_test.select_dtypes(include = ['number']).columns

# Standardizing the numerical columns
scaler = StandardScaler()
clean_train[numeric_columns] = scaler.fit_transform(clean_train[numeric_columns])
# print("Standardized Train DataFrame:\n", clean_train.head(5))
clean_test[numeric_columns_test] = scaler.fit_transform(clean_test[numeric_columns_test])
# print("Standardized Test DataFrame:\n", clean_test.head(5))

# Finding all non-numeric columns
non_numeric_columns = clean_train.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

# Converting categorical features to numerical values using LabelEncoder
label_encoders_train = {}
label_encoders_test = {}
for column in non_numeric_columns:
    le_train = LabelEncoder()
    le_test = LabelEncoder()
    clean_train[column] = le_train.fit_transform(clean_train[column])
    label_encoders_train[column] = le_train
    clean_test[column] = le_test.fit_transform(clean_test[column])
    label_encoders_test[column] = le_test
    
print("Train DataFrame after Label Encoding:\n", clean_train.head(5))

In [ ]:
# Let's see if the label encoding worked.
for col in clean_train.columns:
  if clean_train[col].dtypes == 'object':
    print(col)
  exit()

print("All categorical columns in train data are encoded successfully!")

for col in clean_test.columns:
  if clean_test[col].dtypes == 'object':
    print(col)
  exit()
print("All categorical columns in test data are encoded successfully!")   

In [ ]:
print("Clean train shape: ",clean_train.shape)
print("Clean test shape: ",clean_test.shape)

In [ ]:
# Let's do another check for non numeric columns:
non_numeric_columns_train = clean_train.select_dtypes(include=['object']).columns
non_numeric_columns_test = clean_test.select_dtypes(include=['object']).columns
print("Non-numeric columns in train:\n", non_numeric_columns_train)
print("Non-numeric columns in test:\n", non_numeric_columns_test)

# Feature Selection

Feature selection is the process of identifying and selecting relevant features from large set of features to boost the predictive power and accuracy of the model.

We know that our dataset has a very high dimensionality, this can decrease model efficiency as it will take a long time to train and can also decrease perfomance as it will be prone to overfitting.

Feature selection will help us:
- Decrease the dimension of our dataset
- Speed up machine learning model
- Decrease our likelihood of overfitting.
- Improve ability to comprehend our model results

Let's quickly review the techniques we can use for feature selection:

1.   **Filter Methods**

  These methods select features from the dataset irrespective of the use of any machine learning algorithm by considering the relationship between features and the target variable to compute the importance of features.

  Pros:
  - Fast and inexpensive computation
  - Great for removing duplicated, correlated, redundant features

  Cons:
  - They do not remove multicollinearity. Selection of feature is evaluated individually which can sometimes help when features are in isolation (don’t have a dependency on other features) but will lag when a combination of features can lead to increase in the overall performance of the model.

  Some of the filter methods are:
  - Information Gain – The amount of information provided by the feature for identifying the target value and measures reduction in the entropy values. Information gain of each attribute is calculated considering the target values for feature selection.
  - Chi-square test — Chi-square method (X2) is generally used to test the relationship between categorical variables. It evaluates the independence of a categorical characteristic from the target variable, establishing its relevance and usefulness for classification tasks.
  - Fisher’s Score – Fisher’s Score selects each feature independently according to their scores under Fisher criterion leading to a suboptimal set of features. The larger the Fisher’s score is, the better is the selected feature.
  - Correlation Coefficient – Pearson’s Correlation Coefficient is a measure of quantifying the association between the two continuous variables and the direction of the relationship with its values ranging from -1 to 1.
  - Variance Threshold – It is an approach where all features are removed whose variance doesn’t meet the specific threshold. By default, this method removes features having zero variance. The assumption made using this method is higher variance features are likely to contain more information.
  - Mean Absolute Difference (MAD) – This method is similar to variance threshold method but the difference is there is no square in MAD. This method calculates the mean absolute difference from the mean value.
2.   **Wrapper Methods**

  Wrapper methods also reffered to as also referred to as greedy algorithms,  generate models with a subsets of feature and gauge their model performances. Based on the conclusions made from training in prior to the model, addition and removal of features takes place. Stopping criteria for selecting the best subset are usually pre-defined by the person training the model such as when the performance of the model decreases or a specific number of features has been achieved

  Pro:
  - The main advantage of wrapper methods over the filter methods is that they provide an optimal set of features for training the model, thus resulting in better accuracy than the filter methods.
  
  Con:
  - They are computationally more expensive.

  Some techniques used are:

  - Forward selection – This method is an iterative approach where we initially start with an empty set of features and keep adding a feature which best improves our model after each iteration. The stopping criterion is till the addition of a new variable does not improve the performance of the model.
  - Backward elimination – This method is also an iterative approach where we initially start with all features and after each iteration, we remove the least significant feature. The stopping criterion is till no improvement in the performance of the model is observed after the feature is removed.
  - Bi-directional elimination – This method uses both forward selection and backward elimination technique simultaneously to reach one unique solution.
  Exhaustive selection – This technique is considered as the brute force approach for the evaluation of feature subsets. It creates all possible subsets and builds a learning algorithm for each subset and selects the subset whose model’s performance is best.
  - Recursive elimination – This greedy optimization method selects features by recursively considering the smaller and smaller set of features. The estimator is trained on an initial set of features and their importance is obtained using feature_importance_attribute. The least important features are then removed from the current set of features till we are left with the required number of features.



3.   **Embedded Methods**

  With these, the feature selection algorithm is blended as part of the learning algorithm, thus having its own built-in feature selection methods.
  
  Pro:
  - Are faster like those of filter methods
  - More accurate than the filter methods
  - They take into consideration a combination of features as well.

  Some techniques used are:

  - Regularization – This method adds a penalty to different parameters of the machine learning model to avoid over-fitting of the model. This approach of feature selection uses Lasso (L1 regularization) and Elastic nets (L1 and L2 regularization). The penalty is applied over the coefficients, thus bringing down some coefficients to zero. The features having zero coefficient can be removed from the dataset.
  - Tree-based methods – These methods such as Random Forest, Gradient Boosting,CatBoosting provide us feature importance as a way to select features as well. Feature importance tells us which features are more important in making an impact on the target feature.


After reviewing all of the different feature selction techniques, I am more inclined to use one of the embedded techniques because of their accuracy and speed considering the dimension of my dataset.

I'll be using Lasso Regression.

LASSO stands for Least Absolute Shrinkage And Selection Operator regularization.

Lasso is a form of regularization for linear regression models. Regularization is a statistical method to reduce errors caused by overfitting on training data.

Lasso does feature selection through its L1 penalty term that minimizes the size of all coefficients and allows any coefficient to go to the value of zero, effectively removing input features from the model.

Lasso adds a penalty term to the residual sum of squares (RSS), which is then multiplied by the regularization parameter (lambda or λ). This regularization parameter controls the amount of regularization applied. Larger values of lambda increase the penalty, shrinking more of the coefficients towards zero, which subsequently reduces the importance of (or altogether eliminates) some of the features from the model, which results in automatic feature selection. Conversely, smaller values of lambda reduce the effect of the penalty, retaining more features within the model.


(Geeking a little)The computation of the lasso solution is a quadratic programming problem, and can be tackled by standard numerical analysis algorithms.

In [ ]:
# Since our model is already split into train and test sets, we will split our train data into train and validation then predict on the test set.

# Let's split the data into features and target
X = clean_train.drop('metastatic_diagnosis_period', axis = 1).values
y = clean_train['metastatic_diagnosis_period'].values


# Train Test Split (but really tgrain validation split since we have test set to predict on)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42, stratify = y) 
# When we use an integer for random_state, the function will produce the same results across different executions.
#  The results are only changed if we change the integer value.

print("Shape of Train Features: {}".format(X_train.shape))
print("Shape of Validation Features: {}".format(X_val.shape))
print("Shape of Train Target: {}".format(y_train.shape))
print("Shape of Validation Target: {}".format(y_val.shape))

We'll use **GridSearchCV** to determine optimal hyperparameters for our lasso regression model.

Choosing appropriate values for the alpha parameter in Lasso regression, especially with a large dataset, requires some strategic steps.

The alpha parameter controls the strength of the regularization: a higher value means more regularization, and a lower value means less

Here's a systematic approach to setting up a grid for alpha:

1. Understand the Scale of Your Data:
We standardized our data beforehand because Lasso regression is sensitive to the scale of the features.

2. Initial Exploratory Search:
We'll perform an initial coarse search over a wide range of alpha values using a logarithmic scale. This helps identify a general region where good alpha values might lie.

3. Refine the Search:
Once we identify a promising region, we'll perform a finer search in that range.

In [ ]:
# Using GridSearchCV to find the best hyperparameters.

# Initial coarse GridSearch over a wide range of alpha values
initial_param_grid = {
    'alpha': np.logspace(-6, 6, 13)  # Wide range from 1e-6 to 1e+6
}


# # parameters to be tested on GridSearchCV
# params = {"alpha":np.arange(0.00001, 10, 500)}

# # Number of Folds and adding the random state for replication
# kf=KFold(n_splits=5,shuffle=True, random_state=42)

# Initializing the lasso regression model
lasso = Lasso(max_iter = 30000)

# # GridSearchCV with model, params and folds.
# lasso_cv=GridSearchCV(lasso, param_grid=params, cv=kf)
# lasso_cv.fit(X, y)
# print("Best Params {}".format(lasso_cv.best_params_))



# Setting up our initial GridSearchCV
initial_grid_search = GridSearchCV(estimator = lasso, param_grid = initial_param_grid, cv = 5, scoring = 'neg_mean_squared_error', n_jobs = -1)

# Fitting the model with the initial grid
initial_grid_search.fit(X_train, y_train)

# Getting the best alpha from the initial search
best_initial_alpha = initial_grid_search.best_params_['alpha']
print("Best alpha from initial search:", best_initial_alpha)

# Defining a finer grid around the best alpha from the initial search
fine_param_grid = {
    'alpha': np.linspace(best_initial_alpha / 10, best_initial_alpha * 10, 20)
}

# Setting up fine GridSearchCV
fine_grid_search = GridSearchCV(lasso, fine_param_grid, cv = 5, scoring = 'neg_mean_squared_error', n_jobs = -1)

# Fitting the model with the fine_grid_search
fine_grid_search.fit(X_train, y_train)

# Getting the best parameters and the best score from the fine search
best_params = fine_grid_search.best_params_
best_score = fine_grid_search.best_score_

print("Best Parameters from fine search:", best_params)
print("Best Score from fine search:", best_score)

In [ ]:
# Using Lasso Regressor to plot the best features

# Calling the model with the best parameter
# From the above print statements we got best parameter as {'alpha': 0.006210526315789474}
fine_lasso = Lasso(alpha=0.006210526315789474)
fine_lasso.fit(X_train, y_train)

# Using np.abs() to make coefficients positive.  
fine_lasso_coef = np.abs(fine_lasso.coef_)

feature_names = clean_train.drop('metastatic_diagnosis_period', axis = 1).columns

print("Number of features in the lasso regression: ", fine_lasso.n_features_in_)
# print("Names of features in the lasso regression:\n ", fine_lasso.feature_names_in_)

print(fine_lasso_coef)

# Plotting the Column Names and Importance of Columns. 
plt.bar(feature_names, fine_lasso_coef)
plt.xticks(rotation=90)
plt.grid()
plt.title("Feature Selection Based on Lasso")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.ylim(0, 0.15)
plt.show()

From the plot and print out statement above, we see there are a lot of features with a coefficient of zero. This means they are not providing much value to our model. Let's find out what features are shrunk to 0 and which features were not.

In [ ]:
# let's get the top 20 coefficients in lasso regression and print their respective features
# we can use this to get the name of the features:
# feature_names_in_ndarray of shape (n_features_in_,)
# Names of features seen during fit. Defined only when X has feature names that are all strings.

# and possibly this to get the number of features:
# n_features_in_int
# Number of features seen during fit.

# Here's the documentation
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

In [ ]:
# Create a DataFrame to display feature names and their corresponding coefficients
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': fine_lasso_coef
})

# Sort by the (absolute) value of the coefficient
coef_df = coef_df.sort_values(by='Coefficient', ascending=False)

# Display the top 20 features
top_20_features = coef_df.head(20) # top 20 features that Lasso did not shrink to zero
print("Top 20 features based on absolute coefficient values:\n", top_20_features['Feature'].tolist())

# Display some of the features that lasso shrinked to zero
bottom_20_features = coef_df.tail(20)
print("Some of the features that shrunk to zero:\n", bottom_20_features['Feature'].tolist())


# Plotting the top 20 features with their coefficients
plt.figure(figsize=(10, 8))
plt.barh(top_20_features['Feature'], top_20_features['Coefficient'], color='b')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Top 20 Features by Coefficient Value')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest value at the top
plt.show()

Now we will create a new data set with the top 20 features for model training

In [ ]:
model_df = clean_train[top_20_features['Feature'].tolist()] #minimizing clean_train to only 20 features
print(model_df.shape) #we should see 20 columns

# Model Training

In [ ]:
# We'll be using random forest to train our model so that it can capture non linear relationships
# We'll also do another fit using ridge regression since sometimes trees can lead to overfitting

# Let's split our data again
X_model = model_df # these are our top 20 columns
y_model = clean_train['metastatic_diagnosis_period'].values


# Train Test Split (but really train validation split since we have test set to predict on)
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size = 0.20, random_state = 42, stratify = y) 
# When we use an integer for random_state, the function will produce the same results across different executions.
# The results are only changed if we change the integer value.

print("Shape of Train Features: {}".format(X_train.shape))
print("Shape of Validation Features: {}".format(X_val.shape))
print("Shape of Train Target: {}".format(y_train.shape))
print("Shape of Validation Target: {}".format(y_val.shape))
print("\n")

# Let's create a dictionary with our machine learning models
model_dict = {
    "Ridge": (Ridge(), "regression"),
    "Random Forest Regressor": (RandomForestRegressor(), "regression"),
}
# Let's start with Ridge
ridge = Ridge()
ridge.fit(X_train, y_train) # fitting the model
print("The ridge regressor is successfully fitted:)")

y_pred_ridge = ridge.predict(X_val) #getting predictions

# Performance Metrics for Ridge Regression
print("Performance metrics for Ridge regressor: ")
# Mean Squared Error
ridge_mse = mean_squared_error(y_val, y_pred_ridge)
print("Ridge Mean Squared Error: ", ridge_mse)
# R-squared score
ridge_r2 = r2_score(y_val, y_pred_ridge)
print("Ridge r2 score: ", ridge_r2)
print("\n")


# Now let's run the Random Forest model
forest = RandomForestRegressor(random_state=42,max_depth=6)
forest.fit(X_train, y_train)
print("The random forest regressor is successfully fitted:)")
y_pred_forest = forest.predict(X_val)

# Performance Metrics for Random Forest
print("Performance metrics for Random forest regressor: ")
# Mean Squared Error
forest_mse = mean_squared_error(y_val, y_pred_forest)
print("Forest Mean Squared Error: ", forest_mse)
# R-squared score
forest_r2 = r2_score(y_val, y_pred_forest)
print("Forest r2 score: ", forest_r2)


In [ ]:
plt.hist(y_val)

In [ ]:
plt.hist(y_pred_forest)

From the output above we see that r2 score for random forest is higher than ridge regression. 

Meaning, the forest model has a higher percentage of explaining variance in our independent/outcome variable.

On the other hand, we see that the MSE for forest model is lower than of ridge.

Meaning, a smaller percentage of data points are dispersed widely around the mean in the random forest model compared to ridge model.

Given the 2 comparisons,we'll be picking random forest as our final model. We still have room for more improvement in our forest model, so we'll perform model validation to possibly increase our r2 score and decrease our MSE.

# Model Evaluation and Submission

After seeing how model works by validating our train data, we'll now use the model to predict on the test data.

In [ ]:
# Using random forest model
submission = solution_template.copy()
# Minimizing clean_test to only 20 features
model_test = clean_test[top_20_features['Feature'].tolist()]
# Predicting on test set
submission['metastatic_diagnosis_period'] = forest.predict( model_test)

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
submission['metastatic_diagnosis_period'].hist(bins=100)

We see a surge of frequency at the around the 5o mark. It seems that it takes under 100 days for most of the diagnoses.

# Submission and Notes

Next steps to consider:
* Fine tuning the model
* Using multiple ensemble models
* Creating more visualizations for the 200-250 range for metastatic_diagnosis_period.
